In [2]:
import pandas as pd
import numpy as np
import datetime

In [27]:
# importing game data file with playing condition data 
# from armchairanalysis.com and inspecting headers
gamesaa = pd.read_csv('../data/GAME.csv')
gamesaa.head(1)

,gid,seas,wk,day,v,h,stad,temp,humd,wspd,wdir,cond,surf,ou,sprv,ptsv,ptsh
0,1,2000,1,SUN,SF,ATL,Georgia Dome,79.0,NaN,NaN,NaN,Dome,AstroTurf,42.5,7.0,28,36


In [28]:
gamesaa.shape

(5057, 17)

In [29]:
# importing schedule data file from armchairanalysis.com 
# to get date column and inspecting headers
scheduleaa = pd.read_csv('../data/SCHEDULE.csv')
scheduleaa.head(1)

,gid,seas,wk,day,date,v,h,stad,surf
0,1,2000,1,SUN,9/3/2000,SF,ATL,Georgia Dome,AstroTurf


In [30]:
scheduleaa.shape

(5057, 9)

In [31]:
# reduce schedule file to just the id and date columns
schedule_date = scheduleaa[['gid','date']]
schedule_date.head()

,gid,date
0,1,9/3/2000
1,2,9/3/2000
2,3,9/3/2000
3,4,9/3/2000
4,5,9/3/2000


In [40]:
# inspecting the date column's number format
type(schedule_date['date'][15])

str

In [33]:
# switching the date column format to datetime to change date format
schedule_date['date'] = pd.to_datetime(schedule_date['date'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
# switching date column format back to str format to prepare for merging
schedule_date['date'] = schedule_date['date'].astype('str')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [35]:
# testing the exact location of the date data within the field
schedule_date['date'][15][:10]

'2000-09-10'

In [36]:
# extract date data from above column to create a new 'clean' date column
schedule_date['date2'] = schedule_date['date'].str[:10]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [37]:
# test date new date column
schedule_date['date2'][5000]

'2018-12-15'

In [38]:
# test date new date column
schedule_date.tail(10)

,gid,date,date2
5047,5048,2019-01-05 20:15:00,2019-01-05
5048,5049,2019-01-06 13:05:00,2019-01-06
5049,5050,2019-01-06 16:40:00,2019-01-06
5050,5051,2019-01-12 16:35:00,2019-01-12
5051,5052,2019-01-12 20:15:00,2019-01-12
5052,5053,2019-01-13 13:05:00,2019-01-13
5053,5054,2019-01-13 16:40:00,2019-01-13
5054,5055,2019-01-20 15:05:00,2019-01-20
5055,5056,2019-01-20 18:40:00,2019-01-20
5056,5057,2019-02-03 18:30:00,2019-02-03


In [39]:
# merging games df with newly formated date column
gamesaa_date = pd.merge(gamesaa,schedule_date, how='left', on='gid')
gamesaa_date.head()

,gid,seas,wk,day,v,h,stad,temp,humd,wspd,wdir,cond,surf,ou,sprv,ptsv,ptsh,date,date2
0,1,2000,1,SUN,SF,ATL,Georgia Dome,79.0,NaN,NaN,NaN,Dome,AstroTurf,42.5,7.0,28,36,2000-09-03 00:00:00,2000-09-03
1,2,2000,1,SUN,JAC,CLE,Cleveland Browns Stadium,78.0,63.0,9.0,NE,Sunny,Grass,38.0,-10.0,27,7,2000-09-03 00:00:00,2000-09-03
2,3,2000,1,SUN,PHI,DAL,Texas Stadium,109.0,19.0,5.0,S,Sunny,AstroTurf,40.0,6.0,41,14,2000-09-03 00:00:00,2000-09-03
3,4,2000,1,SUN,NYJ,GB,Lambeau Field,77.0,66.0,5.0,E,Mostly Cloudy,Grass,36.0,2.5,20,16,2000-09-03 00:00:00,2000-09-03
4,5,2000,1,SUN,IND,KC,Arrowhead Stadium,90.0,50.0,8.0,E,Mostly Sunny,Grass,44.0,-3.0,27,14,2000-09-03 00:00:00,2000-09-03


In [42]:
# create unifying ID column called date_id 
# by combining date field with home team field
gamesaa_date['date_id'] = gamesaa_date['date2']+gamesaa_date['h']

In [43]:
# inspecting which columns are useful for new data file
gamesaa_date.columns

Index(['gid', 'seas', 'wk', 'day', 'v', 'h', 'stad', 'temp', 'humd', 'wspd',
       'wdir', 'cond', 'surf', 'ou', 'sprv', 'ptsv', 'ptsh', 'date', 'date2',
       'date_id'],
      dtype='object')

In [44]:
# creating new df with columns related to playing conditions
conditions = gamesaa_date[['gid', 'stad', 'temp', 'humd', 'wspd',
       'wdir', 'cond', 'surf','h','date2', 'date_id']]
conditions.head()

,gid,stad,temp,humd,wspd,wdir,cond,surf,h,date2,date_id
0,1,Georgia Dome,79.0,NaN,NaN,NaN,Dome,AstroTurf,ATL,2000-09-03,2000-09-03ATL
1,2,Cleveland Browns Stadium,78.0,63.0,9.0,NE,Sunny,Grass,CLE,2000-09-03,2000-09-03CLE
2,3,Texas Stadium,109.0,19.0,5.0,S,Sunny,AstroTurf,DAL,2000-09-03,2000-09-03DAL
3,4,Lambeau Field,77.0,66.0,5.0,E,Mostly Cloudy,Grass,GB,2000-09-03,2000-09-03GB
4,5,Arrowhead Stadium,90.0,50.0,8.0,E,Mostly Sunny,Grass,KC,2000-09-03,2000-09-03KC


In [45]:
conditions.shape

(5057, 11)

In [46]:
# saving created file to data folder in csv format
conditions.to_csv('../data/conditions.csv', index=False)